# Conexão com o postgres

* [PostgreSQL Python: Connect To PostgreSQL Database Server](http://www.postgresqltutorial.com/postgresql-python/connect/)
* [PostgreSQL Python: Querying Data](http://www.postgresqltutorial.com/postgresql-python/query/)
* [Stackoverflow - How do I get a list of column names from a psycopg2 cursor?](https://stackoverflow.com/questions/10252247/how-do-i-get-a-list-of-column-names-from-a-psycopg2-cursor)

In [2]:
import pandas as pd
import psycopg2 

Função `config()` recebe os dados de conexão do arquivo `database.ini` como parâmetro e os retorna.

In [ ]:
# Função config recebe o arqui
from configparser import ConfigParser
def config(filename='assets/database.ini', section='postgresql-teste'):
    parser = ConfigParser() # create a parser
    parser.read(filename) # read config file
 
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Seção {0} não encontrada no {1} arquivo'.format(section, filename))
    return db

A função abaixo é apenas de teste, e retorna a versão do banco de dados se tudo estiver certo, e mostra um erro caso houver.

In [ ]:
try:
    params = config()
    print('Conectando no banco de dados')
    conn = psycopg2.connect(**params) # conexão com o banco de dados
    cur = conn.cursor() # cria o cursor
    print('Versão do banco PostgreSQL:')
    cur.execute('SELECT version()')
    db_version = cur.fetchone() # mostra a versão do banco
    print(db_version) 
    cur.close() # encerra a conexão
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Conexão encerrada')

Na função abaixo, conectamos no banco, executamos a query e ela retorna todos os resultados para o dataframe. Em seguida, ela pega os nomes das colunas e envia para o dataframe.

In [ ]:
def get_products():
    """ Query retorna os dados da tabela produtos """
    conn = None
    try:
        params = config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        cur.execute("SELECT * FROM products ORDER BY id ASC LIMIT 10 OFFSET 0")
        df = pd.DataFrame(cur.fetchall()) # insere os dados no dataframe
        colnames = [desc[0] for desc in cur.description] # pega as colunas
        df.columns = colnames
        cur.close() # encerra a conexão
        return df # retorna o dataframe
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [ ]:
base_produtos = get_products()
base_produtos.head()